## توضیحات پروژه
در این پروژه به طراحی و اجرای داشبورد پرداخته شده است

In [31]:
# فراخوانی کتابخانه های مورد استفاده
import pandas as pd
from dash import dcc, html, Dash, callback, Input, Output
import plotly.express as px
import plotly.graph_objects as go

############################## form #############################################
css = ["https://cdn.jsdelivr.net/npm/bootstrap@5.3.1/dist/css/bootstrap.min.css", ]
app = Dash(name="Gapminder Dashboard", external_stylesheets=css)

################### DATASET ####################################
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df = spacex_df.drop("Unnamed: 0",axis=1)

sites = spacex_df["Launch Site"].unique()
DropDown_options=[{'label': 'All Site', 'value': 'All Site'},*[{'label': i, 'value': i} for i in sites]]

min_pyload = spacex_df["Payload Mass (kg)"].min()
max_pyload = spacex_df["Payload Mass (kg)"].max()
#################### CHARTS #####################################
def create_table():
    fig = go.Figure(data=[go.Table(
        header=dict(values=spacex_df.columns, align='left'),
        cells=dict(values=spacex_df.values.T, align='left'))
    ]
    )
    fig.update_layout(paper_bgcolor="#e5ecf6", margin={"t":0, "l":0, "r":0, "b":0}, height=700)
    return fig

def create_pie_chart(enter_site):
        if enter_site == "All Site":
            filtered_df = spacex_df[spacex_df["class"]==1].groupby("Launch Site").count().reset_index()
            fig = px.pie(filtered_df, names="Launch Site", values="class", color="Launch Site",title="Total Launch Success Rate")
            fig.update_layout(paper_bgcolor="#e5ecf6", height=600)
            return fig
        elif enter_site in list(sites):
            filtered_df = spacex_df[spacex_df["Launch Site"]==enter_site].groupby("class").count().reset_index()
            fig = px.pie(filtered_df, names="class", values="Launch Site", color="class",category_orders={"class":[0,1]},
                           title="{} Site, Launch Success Rate".format(enter_site))
            fig.update_layout(paper_bgcolor="#e5ecf6", height=600)
        return fig


def creat_scatter(enter_site,a,b):
        if enter_site == "All Site":
            filter_data = spacex_df[["class","Payload Mass (kg)","Booster Version Category"]]
            pyload_filter = filter_data.groupby("Booster Version Category",group_keys=False).apply(lambda x: x).reset_index()
            fig = px.scatter(pyload_filter,x="Payload Mass (kg)",y="class",color="Booster Version Category",range_x=[a,b+100],title="{} Lunch".format(enter_site))
            fig.update_layout(paper_bgcolor="#e5ecf6", height=400)
            return fig
        elif enter_site in list(sites):
            filtered_df = spacex_df[spacex_df["Launch Site"]==enter_site]
            filter_data = filtered_df[["class","Payload Mass (kg)","Booster Version Category"]]
            pyload_filter = filter_data.groupby("Booster Version Category",group_keys=False).apply(lambda x: x).reset_index()
            fig = px.scatter(pyload_filter,x="Payload Mass (kg)",y="class",color="Booster Version Category",range_x=[a,b+100],title="{} Lunch Site".format(enter_site))
            fig.update_layout(paper_bgcolor="#e5ecf6", height=400)
        return fig

##################### WIDGETS ####################################
site_selection = dcc.Dropdown(id='site-dropdown', options=DropDown_options, value="All Site", placeholder="Select a Launch Site here", searchable=True)

site_selection_scat = dcc.Dropdown(id='site-dropdown_scat', options=DropDown_options, value="All Site", placeholder="Select a Launch Site here", searchable=True)
pyload_selection = dcc.RangeSlider(id="Pyload_mass", min= min_pyload, max=max_pyload,value=[0,10000], step=1000,marks={0: 'minmum',9600: 'maimum'},dots=True)
##################### APP LAYOUT ####################################
app.layout = html.Div([
    html.Div([
        html.H1("Luanch Site Success", className="text-center fw-bold m-2"),
        html.Br(),
        dcc.Tabs([
            dcc.Tab([html.Br(),
                     dcc.Graph(id="dataset", figure=create_table())], label="Dataset"),
            
            dcc.Tab([html.Br(),"Which Site?", site_selection, html.Br(),
                     dcc.Graph(id="pie_chart")], label="Site Success Rate"),
            
            dcc.Tab([html.Br(),"Which Site?", site_selection_scat, html.Br(),
                     html.Br(), "pylod_enter", pyload_selection, html.Br(),
                     dcc.Graph(id="scatter_plot")], label="Payload VS All Sites success "),
        ])
    ], className="col-8 mx-auto"),
], style={"background-color": "#e5ecf6", "height": "100vh"})

##################### CALLBACKS ####################################
@callback(Output("pie_chart", "figure"), [Input("site-dropdown", "value")])
def update_pie_chart(enter_site):
    return create_pie_chart(enter_site)

@callback(Output("scatter_plot", "figure"), [Input("site-dropdown_scat", "value"),Input("Pyload_mass", "value")])
def update_scatter(enter_site,a):
    return creat_scatter(enter_site,a[0],a[1])


if __name__ == "__main__":
    app.run(debug=True,jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
